In [1]:
# Not relying on SLKB 
# Because SLKB uses Day 12 as ETP


%matplotlib inline

#%pylab inline
import pandas as pd
import scipy.stats as stats
import scipy.cluster.hierarchy as clust
import seaborn as sns
import matplotlib.pyplot as plt

%run Horlbeck_Functions.ipynb



C:\Users\Hamda\anaconda3New\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


C:\Users\Hamda\OneDrive\Documents\GitHub\PostDoc\Conway\Research\GIScoring\Hamda's Work\ForGitHub\Horlbeck Scripts
C:\Users\Hamda\OneDrive\Documents\GitHub\PostDoc\Conway\Research\GIScoring\Hamda's Work\ForGitHub\Horlbeck Scripts


In [2]:
reads = pd.read_csv('../InputData/Thompson/raw_counts.txt', sep = "\t")
print(reads.shape)
reads = reads.dropna()  
reads[['sgRNA_guide_seq_g1', 'type' ]] = reads['gRNA'].str.split('_SL_oligo:', expand=True)
reads[['sgRNA_guide_seq_g1', 'sgRNA_guide_seq_g2' ]] = reads['sgRNA_guide_seq_g1'].str.split('_N_', expand=True)

reads.loc[reads['GENE'] == 'Non_Tar_RND_control_Non_Tar_RND_control', 'GENE'] = 'CONTROL_CONTROL'


reads[['sgRNA_target_name_g1', 'sgRNA_target_name_g2']] = reads['GENE'].str.split('_', expand=True)
reads['study_origin'] = 'Thompson'
reads['target_type'] = 'Dual'

#Target_type: Can be ‘Dual’ (two genes targeted),
# can be ‘Single’ (gene + control or gene + gene, where only one gene was targeted with both sgRNAs),
# or ‘Control’ (control + control)
#We also set two guides that target the same sgRNA as ‘Single’ target, such as ‘ABO & ABO’ 
# target would also be classified as a single target, as would ‘ABO & Control’ and ‘Control & ABO’. 
#I don’t know if this would generate the error, but I wanted to mention it nonetheless.
reads.loc[reads['sgRNA_target_name_g1'] == 'Fluc', 'sgRNA_target_name_g1'] = 'CONTROL'
reads.loc[reads['sgRNA_target_name_g2'] == 'Fluc', 'sgRNA_target_name_g2'] = 'CONTROL'

reads.loc[(reads['sgRNA_target_name_g1'] == reads['sgRNA_target_name_g2']), 'target_type'] =   'Single'
reads.loc[(reads['sgRNA_target_name_g1'] == 'CONTROL') | (reads['sgRNA_target_name_g2'] == 'CONTROL'), 'target_type'] = 'Single'
reads.loc[(reads['sgRNA_target_name_g1'] == "CONTROL") & (reads['sgRNA_target_name_g2'] == "CONTROL") , 'target_type'] = "Control"

reads['gene_pair_id'] = reads['sgRNA_target_name_g1'] + "|" + reads['sgRNA_target_name_g2']
reads['gene_pair_orientation'] = reads.apply(lambda row: 'A_B' if row['sgRNA_target_name_g1'] > row['sgRNA_target_name_g2'] else 'B_A', axis=1)
reads['sgRNA_guide_name_g1'] = reads['sgRNA_guide_seq_g1'] 
reads['sgRNA_guide_name_g2'] = reads['sgRNA_guide_seq_g2']
reads

(83686, 23)


,gRNA,GENE,A375_D14_R1,A375_D14_R2,A375_D14_R3,A375_D28_R1,A375_D28_R2,A375_D28_R3,MEWO_D14_R1,MEWO_D14_R2,...,type,sgRNA_guide_seq_g2,sgRNA_target_name_g1,sgRNA_target_name_g2,study_origin,target_type,gene_pair_id,gene_pair_orientation,sgRNA_guide_name_g1,sgRNA_guide_name_g2
1,AARS2_gRNA1_N_AARS_gRNA1_SL_oligo:sl_paralogs_...,AARS2_AARS,272.0,219.0,298.0,8.0,122.0,50.0,433.0,428.0,...,sl_paralogs_to_test,AARS_gRNA1,AARS2,AARS,Thompson,Dual,AARS2|AARS,A_B,AARS2_gRNA1,AARS_gRNA1
3,AARS2_gRNA1_N_AARS_gRNA2_SL_oligo:sl_paralogs_...,AARS2_AARS,217.0,273.0,247.0,98.0,82.0,77.0,557.0,436.0,...,sl_paralogs_to_test,AARS_gRNA2,AARS2,AARS,Thompson,Dual,AARS2|AARS,A_B,AARS2_gRNA1,AARS_gRNA2
5,AARS2_gRNA1_N_AARS_gRNA3_SL_oligo:sl_paralogs_...,AARS2_AARS,216.0,195.0,166.0,74.0,56.0,79.0,589.0,566.0,...,sl_paralogs_to_test,AARS_gRNA3,AARS2,AARS,Thompson,Dual,AARS2|AARS,A_B,AARS2_gRNA1,AARS_gRNA3
7,AARS2_gRNA1_N_AARS_gRNA4_SL_oligo:sl_paralogs_...,AARS2_AARS,24.0,37.0,84.0,0.0,16.0,26.0,317.0,208.0,...,sl_paralogs_to_test,AARS_gRNA4,AARS2,AARS,Thompson,Dual,AARS2|AARS,A_B,AARS2_gRNA1,AARS_gRNA4
9,AARS2_gRNA1_N_AARS_gRNA5_SL_oligo:sl_paralogs_...,AARS2_AARS,35.0,16.0,57.0,36.0,24.0,62.0,82.0,97.0,...,sl_paralogs_to_test,AARS_gRNA5,AARS2,AARS,Thompson,Dual,AARS2|AARS,A_B,AARS2_gRNA1,AARS_gRNA5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83677,ZYX_gRNA5_N_TYMS_gRNA1_SL_oligo:Pos_control_Bi...,ZYX_TYMS,840.0,639.0,625.0,149.0,141.0,112.0,1320.0,1154.0,...,Pos_control_Bisep_Lu_ov_tab_S6sldb_gene_pairs,TYMS_gRNA1,ZYX,TYMS,Thompson,Dual,ZYX|TYMS,A_B,ZYX_gRNA5,TYMS_gRNA1
83679,ZYX_gRNA5_N_TYMS_gRNA2_SL_oligo:Pos_control_Bi...,ZYX_TYMS,749.0,701.0,704.0,803.0,941.0,634.0,798.0,666.0,...,Pos_control_Bisep_Lu_ov_tab_S6sldb_gene_pairs,TYMS_gRNA2,ZYX,TYMS,Thompson,Dual,ZYX|TYMS,A_B,ZYX_gRNA5,TYMS_gRNA2
83681,ZYX_gRNA5_N_TYMS_gRNA3_SL_oligo:Pos_control_Bi...,ZYX_TYMS,547.0,455.0,526.0,128.0,233.0,204.0,833.0,790.0,...,Pos_control_Bisep_Lu_ov_tab_S6sldb_gene_pairs,TYMS_gRNA3,ZYX,TYMS,Thompson,Dual,ZYX|TYMS,A_B,ZYX_gRNA5,TYMS_gRNA3
83683,ZYX_gRNA5_N_TYMS_gRNA4_SL_oligo:Pos_control_Bi...,ZYX_TYMS,949.0,959.0,948.0,1100.0,1436.0,1204.0,979.0,982.0,...,Pos_control_Bisep_Lu_ov_tab_S6sldb_gene_pairs,TYMS_gRNA4,ZYX,TYMS,Thompson,Dual,ZYX|TYMS,A_B,ZYX_gRNA5,TYMS_gRNA4


In [5]:
Thompson_A375 = reads.copy()
Thompson_A375.drop(['MEWO_D14_R2', 'MEWO_D14_R3', 'MEWO_D28_R1', 'MEWO_D28_R2', 'MEWO_D28_R3', 'RPE_D14_R1', 'RPE_D14_R2', 'RPE_D14_R3', 'RPE_D28_R1',
       'RPE_D28_R2', 'RPE_D28_R3', 'A375_D14_R1', 'A375_D14_R2', 'A375_D14_R3'], inplace = True, axis = 1)

Thompson_A375['study_conditions'] = 'Control_R1;Control_R2;Control_R3;A375_D28_R1;A375_D28_R2;A375_D28_R3'
Thompson_A375['gene_pair'] = Thompson_A375['gene_pair_id']
Thompson_A375['count_replicates'] = Thompson_A375['Control_R1'].astype(str) + ";" + Thompson_A375['Control_R2'].astype(str) + ";" + Thompson_A375['Control_R3'].astype(str) + ";" +  Thompson_A375['A375_D28_R1'].astype(str) + ";" + Thompson_A375['A375_D28_R2'].astype(str) + ";" + Thompson_A375['A375_D28_R3'].astype(str) 
Thompson_A375['cell_line_origin'] = "A375"
Thompson_A375['T0_counts'] = Thompson_A375['Control_R1'].astype(str) + ";" + Thompson_A375['Control_R2'].astype(str) + ";" + Thompson_A375['Control_R3'].astype(str)
Thompson_A375['T0_replicate_names'] =  'Control_R1;Control_R2;Control_R3'
Thompson_A375['TEnd_counts'] = Thompson_A375['A375_D28_R1'].astype(str) + ";" + Thompson_A375['A375_D28_R2'].astype(str) + ";" + Thompson_A375['A375_D28_R3'].astype(str)
Thompson_A375['TEnd_replicate_names'] = "A375_D28_R1;A375_D28_R2;A375_D28_R3"
Thompson_A375 = Thompson_A375[['sgRNA_guide_name_g1','sgRNA_guide_name_g2','sgRNA_target_name_g1','sgRNA_target_name_g2','count_replicates','cell_line_origin','study_conditions','study_origin','target_type','T0_counts','T0_replicate_names','TEnd_counts','TEnd_replicate_names','gene_pair','gene_pair_orientation']]
Thompson_A375

Thompson_A375 = run_horlbeck_score(Thompson_A375, 'Thompson', "A375", do_preprocessing= True, re_run=True)
Thompson_A375['HORLBECK_SCORE'].to_csv("Horlbeck Output/Thompson_Horlbeck_A375.csv")


,sgRNA_guide_name_g1,sgRNA_guide_name_g2,sgRNA_target_name_g1,sgRNA_target_name_g2,count_replicates,cell_line_origin,study_conditions,study_origin,target_type,T0_counts,T0_replicate_names,TEnd_counts,TEnd_replicate_names,gene_pair,gene_pair_orientation
1,AARS2_gRNA1,AARS_gRNA1,AARS2,AARS,509.0;442.0;411.0;8.0;122.0;50.0,A375,Control_R1;Control_R2;Control_R3;A375_D28_R1;A...,Thompson,Dual,509.0;442.0;411.0,Control_R1;Control_R2;Control_R3,8.0;122.0;50.0,A375_D28_R1;A375_D28_R2;A375_D28_R3,AARS2|AARS,A_B
3,AARS2_gRNA1,AARS_gRNA2,AARS2,AARS,530.0;642.0;501.0;98.0;82.0;77.0,A375,Control_R1;Control_R2;Control_R3;A375_D28_R1;A...,Thompson,Dual,530.0;642.0;501.0,Control_R1;Control_R2;Control_R3,98.0;82.0;77.0,A375_D28_R1;A375_D28_R2;A375_D28_R3,AARS2|AARS,A_B
5,AARS2_gRNA1,AARS_gRNA3,AARS2,AARS,804.0;663.0;633.0;74.0;56.0;79.0,A375,Control_R1;Control_R2;Control_R3;A375_D28_R1;A...,Thompson,Dual,804.0;663.0;633.0,Control_R1;Control_R2;Control_R3,74.0;56.0;79.0,A375_D28_R1;A375_D28_R2;A375_D28_R3,AARS2|AARS,A_B
7,AARS2_gRNA1,AARS_gRNA4,AARS2,AARS,711.0;687.0;622.0;0.0;16.0;26.0,A375,Control_R1;Control_R2;Control_R3;A375_D28_R1;A...,Thompson,Dual,711.0;687.0;622.0,Control_R1;Control_R2;Control_R3,0.0;16.0;26.0,A375_D28_R1;A375_D28_R2;A375_D28_R3,AARS2|AARS,A_B
9,AARS2_gRNA1,AARS_gRNA5,AARS2,AARS,602.0;752.0;569.0;36.0;24.0;62.0,A375,Control_R1;Control_R2;Control_R3;A375_D28_R1;A...,Thompson,Dual,602.0;752.0;569.0,Control_R1;Control_R2;Control_R3,36.0;24.0;62.0,A375_D28_R1;A375_D28_R2;A375_D28_R3,AARS2|AARS,A_B
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83677,ZYX_gRNA5,TYMS_gRNA1,ZYX,TYMS,1036.0;1279.0;1098.0;149.0;141.0;112.0,A375,Control_R1;Control_R2;Control_R3;A375_D28_R1;A...,Thompson,Dual,1036.0;1279.0;1098.0,Control_R1;Control_R2;Control_R3,149.0;141.0;112.0,A375_D28_R1;A375_D28_R2;A375_D28_R3,ZYX|TYMS,A_B
83679,ZYX_gRNA5,TYMS_gRNA2,ZYX,TYMS,698.0;687.0;455.0;803.0;941.0;634.0,A375,Control_R1;Control_R2;Control_R3;A375_D28_R1;A...,Thompson,Dual,698.0;687.0;455.0,Control_R1;Control_R2;Control_R3,803.0;941.0;634.0,A375_D28_R1;A375_D28_R2;A375_D28_R3,ZYX|TYMS,A_B
83681,ZYX_gRNA5,TYMS_gRNA3,ZYX,TYMS,646.0;746.0;679.0;128.0;233.0;204.0,A375,Control_R1;Control_R2;Control_R3;A375_D28_R1;A...,Thompson,Dual,646.0;746.0;679.0,Control_R1;Control_R2;Control_R3,128.0;233.0;204.0,A375_D28_R1;A375_D28_R2;A375_D28_R3,ZYX|TYMS,A_B
83683,ZYX_gRNA5,TYMS_gRNA4,ZYX,TYMS,827.0;653.0;606.0;1100.0;1436.0;1204.0,A375,Control_R1;Control_R2;Control_R3;A375_D28_R1;A...,Thompson,Dual,827.0;653.0;606.0,Control_R1;Control_R2;Control_R3,1100.0;1436.0;1204.0,A375_D28_R1;A375_D28_R2;A375_D28_R3,ZYX|TYMS,A_B


In [7]:
Thompson_MEWO = reads.copy() # Only keep MEWO D28
Thompson_MEWO.drop(['MEWO_D14_R1','MEWO_D14_R2', 'MEWO_D14_R3', 'RPE_D14_R1', 'RPE_D14_R2', 'RPE_D14_R3', 'RPE_D28_R1',
       'RPE_D28_R2', 'RPE_D28_R3', 'A375_D14_R1', 'A375_D14_R2', 'A375_D14_R3', 'A375_D28_R1', 'A375_D28_R2', 'A375_D28_R3'], inplace = True, axis = 1)

Thompson_MEWO['study_conditions'] = 'Control_R1;Control_R2;Control_R3;MEWO_D28_R1;MEWO_D28_R2;MEWO_D28_R3'
Thompson_MEWO['gene_pair'] = Thompson_MEWO['gene_pair_id']
Thompson_MEWO['count_replicates'] = Thompson_MEWO['Control_R1'].astype(str) + ";" + Thompson_MEWO['Control_R2'].astype(str) + ";" + Thompson_MEWO['Control_R3'].astype(str) + ";" +  Thompson_MEWO['MEWO_D28_R1'].astype(str) + ";" + Thompson_MEWO['MEWO_D28_R2'].astype(str) + ";" + Thompson_MEWO['MEWO_D28_R3'].astype(str) 
Thompson_MEWO['cell_line_origin'] = "MEWO"
Thompson_MEWO['T0_counts'] = Thompson_MEWO['Control_R1'].astype(str) + ";" + Thompson_MEWO['Control_R2'].astype(str) + ";" + Thompson_MEWO['Control_R3'].astype(str)
Thompson_MEWO['T0_replicate_names'] =  'Control_R1;Control_R2;Control_R3'
Thompson_MEWO['TEnd_counts'] = Thompson_MEWO['MEWO_D28_R1'].astype(str) + ";" + Thompson_MEWO['MEWO_D28_R2'].astype(str) + ";" + Thompson_MEWO['MEWO_D28_R3'].astype(str)
Thompson_MEWO['TEnd_replicate_names'] = "MEWO_D28_R1;MEWO_D28_R2;MEWO_D28_R3"
Thompson_MEWO = Thompson_MEWO[['sgRNA_guide_name_g1','sgRNA_guide_name_g2','sgRNA_target_name_g1','sgRNA_target_name_g2','count_replicates','cell_line_origin','study_conditions','study_origin','target_type','T0_counts','T0_replicate_names','TEnd_counts','TEnd_replicate_names','gene_pair','gene_pair_orientation']]
Thompson_MEWO

Thompson_MEWO = run_horlbeck_score(Thompson_MEWO, 'Thompson', "MEWO", do_preprocessing= True, re_run=True, filterThreshold=0)
Thompson_MEWO['HORLBECK_SCORE'].to_csv("Horlbeck Output/Thompson_Horlbeck_MEWO.csv")
Thompson_MEWO

,sgRNA_guide_name_g1,sgRNA_guide_name_g2,sgRNA_target_name_g1,sgRNA_target_name_g2,count_replicates,cell_line_origin,study_conditions,study_origin,target_type,T0_counts,T0_replicate_names,TEnd_counts,TEnd_replicate_names,gene_pair,gene_pair_orientation
1,AARS2_gRNA1,AARS_gRNA1,AARS2,AARS,509.0;442.0;411.0;76.0;67.0;118.0,MEWO,Control_R1;Control_R2;Control_R3;MEWO_D28_R1;M...,Thompson,Dual,509.0;442.0;411.0,Control_R1;Control_R2;Control_R3,76.0;67.0;118.0,MEWO_D28_R1;MEWO_D28_R2;MEWO_D28_R3,AARS2|AARS,A_B
3,AARS2_gRNA1,AARS_gRNA2,AARS2,AARS,530.0;642.0;501.0;254.0;83.0;103.0,MEWO,Control_R1;Control_R2;Control_R3;MEWO_D28_R1;M...,Thompson,Dual,530.0;642.0;501.0,Control_R1;Control_R2;Control_R3,254.0;83.0;103.0,MEWO_D28_R1;MEWO_D28_R2;MEWO_D28_R3,AARS2|AARS,A_B
5,AARS2_gRNA1,AARS_gRNA3,AARS2,AARS,804.0;663.0;633.0;100.0;48.0;123.0,MEWO,Control_R1;Control_R2;Control_R3;MEWO_D28_R1;M...,Thompson,Dual,804.0;663.0;633.0,Control_R1;Control_R2;Control_R3,100.0;48.0;123.0,MEWO_D28_R1;MEWO_D28_R2;MEWO_D28_R3,AARS2|AARS,A_B
7,AARS2_gRNA1,AARS_gRNA4,AARS2,AARS,711.0;687.0;622.0;91.0;61.0;89.0,MEWO,Control_R1;Control_R2;Control_R3;MEWO_D28_R1;M...,Thompson,Dual,711.0;687.0;622.0,Control_R1;Control_R2;Control_R3,91.0;61.0;89.0,MEWO_D28_R1;MEWO_D28_R2;MEWO_D28_R3,AARS2|AARS,A_B
9,AARS2_gRNA1,AARS_gRNA5,AARS2,AARS,602.0;752.0;569.0;37.0;2.0;66.0,MEWO,Control_R1;Control_R2;Control_R3;MEWO_D28_R1;M...,Thompson,Dual,602.0;752.0;569.0,Control_R1;Control_R2;Control_R3,37.0;2.0;66.0,MEWO_D28_R1;MEWO_D28_R2;MEWO_D28_R3,AARS2|AARS,A_B
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83677,ZYX_gRNA5,TYMS_gRNA1,ZYX,TYMS,1036.0;1279.0;1098.0;887.0;889.0;829.0,MEWO,Control_R1;Control_R2;Control_R3;MEWO_D28_R1;M...,Thompson,Dual,1036.0;1279.0;1098.0,Control_R1;Control_R2;Control_R3,887.0;889.0;829.0,MEWO_D28_R1;MEWO_D28_R2;MEWO_D28_R3,ZYX|TYMS,A_B
83679,ZYX_gRNA5,TYMS_gRNA2,ZYX,TYMS,698.0;687.0;455.0;1003.0;626.0;637.0,MEWO,Control_R1;Control_R2;Control_R3;MEWO_D28_R1;M...,Thompson,Dual,698.0;687.0;455.0,Control_R1;Control_R2;Control_R3,1003.0;626.0;637.0,MEWO_D28_R1;MEWO_D28_R2;MEWO_D28_R3,ZYX|TYMS,A_B
83681,ZYX_gRNA5,TYMS_gRNA3,ZYX,TYMS,646.0;746.0;679.0;877.0;655.0;499.0,MEWO,Control_R1;Control_R2;Control_R3;MEWO_D28_R1;M...,Thompson,Dual,646.0;746.0;679.0,Control_R1;Control_R2;Control_R3,877.0;655.0;499.0,MEWO_D28_R1;MEWO_D28_R2;MEWO_D28_R3,ZYX|TYMS,A_B
83683,ZYX_gRNA5,TYMS_gRNA4,ZYX,TYMS,827.0;653.0;606.0;1080.0;1149.0;791.0,MEWO,Control_R1;Control_R2;Control_R3;MEWO_D28_R1;M...,Thompson,Dual,827.0;653.0;606.0,Control_R1;Control_R2;Control_R3,1080.0;1149.0;791.0,MEWO_D28_R1;MEWO_D28_R2;MEWO_D28_R3,ZYX|TYMS,A_B


In [8]:
Thompson_RPE1 = reads.copy() # Only keep MEWO D28
Thompson_RPE1.drop(['MEWO_D14_R1','MEWO_D14_R2', 'MEWO_D14_R3', 'RPE_D14_R1', 'RPE_D14_R2', 'RPE_D14_R3', 'MEWO_D28_R1',
       'MEWO_D28_R2', 'MEWO_D28_R3', 'A375_D14_R1', 'A375_D14_R2', 'A375_D14_R3', 'A375_D28_R1', 'A375_D28_R2', 'A375_D28_R3'], inplace = True, axis = 1)
Thompson_RPE1['study_conditions'] = 'Control_R1;Control_R2;Control_R3;RPE1_D28_R1;RPE1_D28_R2;RPE1_D28_R3'
Thompson_RPE1['gene_pair'] = Thompson_RPE1['gene_pair_id']
Thompson_RPE1['count_replicates'] = Thompson_RPE1['Control_R1'].astype(str) + ";" + Thompson_RPE1['Control_R2'].astype(str) + ";" + Thompson_RPE1['Control_R3'].astype(str) + ";" +  Thompson_RPE1['RPE_D28_R1'].astype(str) + ";" + Thompson_RPE1['RPE_D28_R2'].astype(str) + ";" + Thompson_RPE1['RPE_D28_R3'].astype(str) 
Thompson_RPE1['cell_line_origin'] = "RPE1"
Thompson_RPE1['T0_counts'] = Thompson_RPE1['Control_R1'].astype(str) + ";" + Thompson_RPE1['Control_R2'].astype(str) + ";" + Thompson_RPE1['Control_R3'].astype(str)
Thompson_RPE1['T0_replicate_names'] =  'Control_R1;Control_R2;Control_R3'
Thompson_RPE1['TEnd_counts'] = Thompson_RPE1['RPE_D28_R1'].astype(str) + ";" + Thompson_RPE1['RPE_D28_R2'].astype(str) + ";" + Thompson_RPE1['RPE_D28_R3'].astype(str)
Thompson_RPE1['TEnd_replicate_names'] = "RPE_D28_R1;RPE_D28_R2;RPE_D28_R3"
Thompson_RPE1 = Thompson_RPE1[['sgRNA_guide_name_g1','sgRNA_guide_name_g2','sgRNA_target_name_g1','sgRNA_target_name_g2','count_replicates','cell_line_origin','study_conditions','study_origin','target_type','T0_counts','T0_replicate_names','TEnd_counts','TEnd_replicate_names','gene_pair','gene_pair_orientation']]
Thompson_RPE1

Thompson_RPE1 = run_horlbeck_score(Thompson_RPE1, 'Thompson', "RPE1", do_preprocessing= True, re_run=True, filterThreshold=0)
Thompson_RPE1['HORLBECK_SCORE'].to_csv("Horlbeck Output/Thompson_Horlbeck_RPE1.csv")
Thompson_RPE1

,sgRNA_guide_name_g1,sgRNA_guide_name_g2,sgRNA_target_name_g1,sgRNA_target_name_g2,count_replicates,cell_line_origin,study_conditions,study_origin,target_type,T0_counts,T0_replicate_names,TEnd_counts,TEnd_replicate_names,gene_pair,gene_pair_orientation
1,AARS2_gRNA1,AARS_gRNA1,AARS2,AARS,509.0;442.0;411.0;71.0;54.0;43.0,RPE1,Control_R1;Control_R2;Control_R3;RPE1_D28_R1;R...,Thompson,Dual,509.0;442.0;411.0,Control_R1;Control_R2;Control_R3,71.0;54.0;43.0,RPE_D28_R1;RPE_D28_R2;RPE_D28_R3,AARS2|AARS,A_B
3,AARS2_gRNA1,AARS_gRNA2,AARS2,AARS,530.0;642.0;501.0;14.0;6.0;10.0,RPE1,Control_R1;Control_R2;Control_R3;RPE1_D28_R1;R...,Thompson,Dual,530.0;642.0;501.0,Control_R1;Control_R2;Control_R3,14.0;6.0;10.0,RPE_D28_R1;RPE_D28_R2;RPE_D28_R3,AARS2|AARS,A_B
5,AARS2_gRNA1,AARS_gRNA3,AARS2,AARS,804.0;663.0;633.0;4.0;26.0;23.0,RPE1,Control_R1;Control_R2;Control_R3;RPE1_D28_R1;R...,Thompson,Dual,804.0;663.0;633.0,Control_R1;Control_R2;Control_R3,4.0;26.0;23.0,RPE_D28_R1;RPE_D28_R2;RPE_D28_R3,AARS2|AARS,A_B
7,AARS2_gRNA1,AARS_gRNA4,AARS2,AARS,711.0;687.0;622.0;15.0;50.0;0.0,RPE1,Control_R1;Control_R2;Control_R3;RPE1_D28_R1;R...,Thompson,Dual,711.0;687.0;622.0,Control_R1;Control_R2;Control_R3,15.0;50.0;0.0,RPE_D28_R1;RPE_D28_R2;RPE_D28_R3,AARS2|AARS,A_B
9,AARS2_gRNA1,AARS_gRNA5,AARS2,AARS,602.0;752.0;569.0;28.0;26.0;21.0,RPE1,Control_R1;Control_R2;Control_R3;RPE1_D28_R1;R...,Thompson,Dual,602.0;752.0;569.0,Control_R1;Control_R2;Control_R3,28.0;26.0;21.0,RPE_D28_R1;RPE_D28_R2;RPE_D28_R3,AARS2|AARS,A_B
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83677,ZYX_gRNA5,TYMS_gRNA1,ZYX,TYMS,1036.0;1279.0;1098.0;293.0;449.0;582.0,RPE1,Control_R1;Control_R2;Control_R3;RPE1_D28_R1;R...,Thompson,Dual,1036.0;1279.0;1098.0,Control_R1;Control_R2;Control_R3,293.0;449.0;582.0,RPE_D28_R1;RPE_D28_R2;RPE_D28_R3,ZYX|TYMS,A_B
83679,ZYX_gRNA5,TYMS_gRNA2,ZYX,TYMS,698.0;687.0;455.0;350.0;456.0;323.0,RPE1,Control_R1;Control_R2;Control_R3;RPE1_D28_R1;R...,Thompson,Dual,698.0;687.0;455.0,Control_R1;Control_R2;Control_R3,350.0;456.0;323.0,RPE_D28_R1;RPE_D28_R2;RPE_D28_R3,ZYX|TYMS,A_B
83681,ZYX_gRNA5,TYMS_gRNA3,ZYX,TYMS,646.0;746.0;679.0;167.0;230.0;326.0,RPE1,Control_R1;Control_R2;Control_R3;RPE1_D28_R1;R...,Thompson,Dual,646.0;746.0;679.0,Control_R1;Control_R2;Control_R3,167.0;230.0;326.0,RPE_D28_R1;RPE_D28_R2;RPE_D28_R3,ZYX|TYMS,A_B
83683,ZYX_gRNA5,TYMS_gRNA4,ZYX,TYMS,827.0;653.0;606.0;303.0;202.0;453.0,RPE1,Control_R1;Control_R2;Control_R3;RPE1_D28_R1;R...,Thompson,Dual,827.0;653.0;606.0,Control_R1;Control_R2;Control_R3,303.0;202.0;453.0,RPE_D28_R1;RPE_D28_R2;RPE_D28_R3,ZYX|TYMS,A_B
